In [1]:
import numpy as np

from pycox import datasets
from lifelines.datasets import load_rossi
from sksurv.datasets import (
    load_aids,
    load_breast_cancer,
    load_flchain,
    load_gbsg2,
    load_whas500,
)
from sklearn.preprocessing import LabelEncoder


def get_dataset(name: str):
    if name == "metabric":
        df = datasets.metabric.read_df()
    elif name == "support":
        df = datasets.support.read_df()
    elif name == "gbsg":
        df = datasets.gbsg.read_df()
    elif name == "rossi":
        df = load_rossi()
        df = df.rename(columns={"week": "duration", "arrest": "event"})
    elif name == "aids":
        X, Y = load_aids()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "flchain":
        X, Y = load_flchain()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "gbsg2":
        X, Y = load_gbsg2()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "whas500":
        X, Y = load_whas500()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]

    for col in df.columns:
        if df[col].dtype.name in ["object", "category"]:
            df[col] = LabelEncoder().fit_transform(df[col])

    duration_col = "duration"
    event_col = "event"
    
    df = df[df["duration"] > 0].fillna(0)

    X = df.drop(columns=[duration_col, event_col])
    Y = df[event_col]
    T = df[duration_col]

    time_horizons = np.linspace(T.min(), T.max(), num=5)[1:-1]

    return df, X, T, Y, time_horizons

In [2]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from typing import Tuple
import optuna

from synthcity.plugins.models.time_to_event.tte_aft import WeibullAFTTimeToEvent
from synthcity.plugins.models.time_to_event.tte_coxph import CoxPHTimeToEvent
from synthcity.plugins.models.time_to_event.tte_rsf import RandomSurvivalForestTimeToEvent
from synthcity.plugins.models.time_to_event.tte_xgb import XGBTimeToEvent
from synthcity.plugins.models.time_to_event.tte_deephit import DeephitTimeToEvent
from synthcity.plugins.models.time_to_event.tte_date import DATETimeToEvent
from synthcity.plugins.models.time_to_event.tte_robust_date import RobustDATETimeToEvent
from synthcity.plugins.models.time_to_event.tte_tenn import TENNTimeToEvent

from synthcity.plugins.models.time_to_event.metrics import (
     expected_time_error,
     ranking_error,
     rush_error,
     c_index,
 )
optuna.logging.set_verbosity(optuna.logging.DEBUG)
 

def generate_score(metric: np.ndarray) -> Tuple[float, float]:
    percentile_val = 1.96
    score = (np.mean(metric), percentile_val * np.std(metric) / np.sqrt(len(metric)))
 
    return str(round(score[0], 4)) + " +/- " + str(round(score[1], 4))
    

def models():
    return [
        "tenn",  
        #"date",
        #"weibull_aft", 
        #"cox_ph", 
        #"random_survival_forest", 
        #"survival_xgboost", 
        #"deephit", 
    ]

def _trial_params(trial, param_space):
    out = {}
    
    for param in param_space:
        if hasattr(param, 'choices'):
            out[param.name] = trial.suggest_categorical(param.name, choices = param.choices)
        elif hasattr(param, 'step'):
            out[param.name] = trial.suggest_int(param.name, param.low, param.high, param.step)
        else:
            out[param.name] = trial.suggest_float(param.name, param.low, param.high)
            
    return out

def objective_meta(X_train, T_train, E_train, X_test, T_test, E_test, template):
    def objective(trial):
        args = _trial_params(trial, template.hyperparameter_space())
        try:
            model = template(**args).fit(X_train, T_train, E_train)
            preds = model.predict(X_test)
        except BaseException as e:
            print("model search failed", template.name(), e)
            return 0
    
        #return expected_time_error(T_test, E_test, preds) - c_index(T_test, E_test, preds)
        return c_index(T_test, E_test, preds)

    return objective

def search_best_args(study_name: str, X_train, T_train, E_train, X_test, T_test, E_test, template):
    storage = optuna.storages.RedisStorage(url="redis://localhost")
    study = optuna.create_study(
        study_name = study_name, 
        direction='maximize',
        storage = storage,
        load_if_exists = True,
    )
    study.optimize(objective_meta(X_train, T_train, E_train, X_test, T_test, E_test, template), n_trials=100)
    
    if study.best_trial.params is not None:
        return study.best_trial.params
        
    return {}

def get_default_args(model):
    defaults = {
        "tenn" : {
            'n_layers_hidden': 1, 
            'n_units_hidden': 350, 
            'nonlin': 'relu', 
            'n_iter': 1000, 
            'dropout': 0.02506611632175633, 
            'lr': 0.0001, 
            'batch_size': 250, 
            'lambda_calibration': 1, 
            'lambda_regression_nc': 1, 
            'lambda_regression_c': 1, 
            'lambda_ordering': 1,
        },
        "date": {
            'generator_n_layers_hidden': 2, 
            'generator_n_units_hidden': 250, 
            'generator_nonlin': 'relu', 
            'generator_n_iter': 2000, 
            'generator_dropout': 0.10619582443953746, 
            'generator_lr': 0.0001, 
            'discriminator_n_layers_hidden': 5, 
            'discriminator_n_units_hidden': 300, 
            'discriminator_nonlin': 'relu', 
            'discriminator_dropout': 0.01592627865323451, 
            'discriminator_lr': 0.0001, 
            'batch_size': 1000
        },
        "weibull_aft" : {
            'alpha': 0.043203641175350804, 
            'l1_ratio': 0.12034835856558748
        },
        "cox_ph": {
            'alpha': 0.0859489463050206, 
            'penalizer': 0.002271000821316366
        },
        "random_survival_forest": {},
        "survival_xgboost" : {
            'max_depth': 4, 
            'min_child_weight': 26, 
            'objective': 'aft', 
            'strategy': 'debiased_bce'
        },
        "deephit": {
            "num_duration": 500,
            'batch_size': 500, 
            'lr': 0.01, 
            'dim_hidden': 20, 
            'alpha': 0.005033592098894424, 
            'sigma': 0.45827704983746737, 
            'dropout': 0.12545190397806258, 
            'patience': 44
        },
    }
    
    if model in defaults:
        return defaults[model]
    
    return {}


def get_model_template(model):
    defaults = {
        "tenn" : TENNTimeToEvent,
        "date": DATETimeToEvent,
        "weibull_aft" : WeibullAFTTimeToEvent,
        "cox_ph": CoxPHTimeToEvent,
        "random_survival_forest": RandomSurvivalForestTimeToEvent,
        "survival_xgboost" : XGBTimeToEvent,
        "deephit": DeephitTimeToEvent,
    }
    
    if model in defaults:
        return defaults[model]
    
    raise RuntimeError(f"invalid model model")

def evaluate_model(
              model_name,
              model_args,
              X: pd.DataFrame, 
              T: pd.DataFrame, 
              E: pd.DataFrame, 
              n_folds: int = 3, 
              seed: int = 0,
             ):
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    local_results = {
                "te_err_l1_ood" : [],
                "te_err_l2_ood" : [],
                "c_index_ood" : [],
                "te_err_l1_id" : [],
                "te_err_l2_id" : [],
                "c_index_id" : [],
            }

    model_template = get_model_template(model_name)

    for train_index, test_index in skf.split(X, E):
        X_train = X.loc[X.index[train_index]]
        E_train = E.loc[E.index[train_index]]
        T_train = T.loc[T.index[train_index]]

        X_test = X.loc[X.index[test_index]]
        E_test = E.loc[E.index[test_index]]
        T_test = T.loc[T.index[test_index]]

        model = model_template(**model_args)

        try:
            model.fit(X_train, T_train, E_train)
            ood_preds = model.predict(X_test)
            id_preds = model.predict(X_train)

        except BaseException as e:
            print("Model training failed", model.name(), args, e)
            continue

        local_results["te_err_l1_ood"].append(expected_time_error(T_test, E_test, ood_preds, metric = "l1"))
        local_results["te_err_l2_ood"].append(expected_time_error(T_test, E_test, ood_preds, metric = "l2"))
        local_results["c_index_ood"].append(c_index(T_test, E_test, ood_preds))

        local_results["te_err_l1_id"].append(expected_time_error(T_train, E_train, id_preds, metric = "l1"))
        local_results["te_err_l2_id"].append(expected_time_error(T_train, E_train, id_preds, metric = "l2"))
        local_results["c_index_id"].append(c_index(T_train, E_train, id_preds))

    return local_results

def benchmark_tte_reconstruction(
              study_name : str,
              X: pd.DataFrame, 
              T: pd.DataFrame, 
              E: pd.DataFrame, 
              n_folds: int = 3, 
              seed: int = 0,
              automl: bool = False,
             ):
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    raw_results = {}
    headers = ["Model", "TE Error L1(OOD)", "TE Error L2(OOD)", "C-Index(OOD)", 
               "TE Error L1(ID)", "TE Error L2(ID)", "C-Index(ID)"]
    results = []
    
    idx = 0
    for model_name in models():
        local_results = {
                    "te_err_l1_ood" : [],
                    "te_err_l2_ood" : [],
                    "c_index_ood" : [],
                    "te_err_l1_id" : [],
                    "te_err_l2_id" : [],
                    "c_index_id" : [],
                }

        args = get_default_args(model_name)
            
        local_results = evaluate_model(model_name, args, X, T, E, n_folds = n_folds, seed = seed)

        raw_results[model_name] = local_results

    for model in raw_results:
        te_err_l1_ood = generate_score(raw_results[model]["te_err_l1_ood"])
        te_err_l2_ood = generate_score(raw_results[model]["te_err_l2_ood"])
        c_index_ood = generate_score(raw_results[model]["c_index_ood"])
        te_err_l1_id = generate_score(raw_results[model]["te_err_l1_id"])
        te_err_l2_id = generate_score(raw_results[model]["te_err_l2_id"])
        c_index_id = generate_score(raw_results[model]["c_index_id"])
        
        results.append([model, te_err_l1_ood, te_err_l2_ood, c_index_ood,
                        te_err_l1_id, te_err_l2_id, c_index_id])
    
    return pd.DataFrame(results, columns = headers)

## AIDS dataset

In [3]:
import pandas as pd

df, X_real, T_real, Y_real, time_horizons = get_dataset("aids")

X = df.drop(columns=["event", "duration"])
E = df["event"]
T = df["duration"]

X

,age,cd4,hemophil,ivdrug,karnof,priorzdv,raceth,sex,strat2,tx,txgrp
0,34.0,169.0,0,0,0,39.0,0,0,1,0,0
1,34.0,149.5,0,0,3,15.0,1,1,1,0,0
2,20.0,23.5,1,0,0,9.0,0,0,0,1,1
3,48.0,46.0,0,0,3,53.0,0,0,1,0,0
4,46.0,10.0,0,2,3,12.0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1146,44.0,65.5,0,0,0,103.0,0,0,1,1,1
1147,41.0,7.5,0,0,2,20.0,1,0,0,1,1
1148,43.0,170.0,0,2,3,27.0,1,0,1,0,0
1149,44.0,282.5,0,2,2,12.0,0,0,1,0,0


In [4]:
metrics = benchmark_tte_reconstruction("aids", X, T, E)

In [5]:
metrics

,Model,TE Error L1(OOD),TE Error L2(OOD),C-Index(OOD),TE Error L1(ID),TE Error L2(ID),C-Index(ID)
0,tenn,3.91 +/- 0.3248,4.6261 +/- 0.3019,0.7146 +/- 0.0351,2.3236 +/- 0.1342,2.6438 +/- 0.1304,0.8539 +/- 0.0207
1,date,3.6876 +/- 0.5182,4.4133 +/- 0.4914,0.7181 +/- 0.0431,2.7867 +/- 0.1999,3.2788 +/- 0.1856,0.8082 +/- 0.0345
2,weibull_aft,6.0072 +/- 0.2411,6.4062 +/- 0.2252,0.738 +/- 0.0622,5.9278 +/- 0.1926,6.3398 +/- 0.1529,0.7631 +/- 0.0305
3,cox_ph,6.0101 +/- 0.227,6.409 +/- 0.2154,0.7383 +/- 0.0622,5.9348 +/- 0.1863,6.3458 +/- 0.1504,0.7625 +/- 0.0302
4,random_survival_forest,4.3953 +/- 0.2455,4.8763 +/- 0.2448,0.7326 +/- 0.0267,3.2524 +/- 0.1241,3.7066 +/- 0.1483,0.9722 +/- 0.002
5,survival_xgboost,4.5924 +/- 0.1979,5.0506 +/- 0.2304,0.7468 +/- 0.0417,4.5378 +/- 0.0722,4.9962 +/- 0.0794,0.7867 +/- 0.0236
6,deephit,3.5082 +/- 0.5359,4.0373 +/- 0.5009,0.73 +/- 0.0425,3.2987 +/- 0.8446,3.9138 +/- 0.7756,0.7404 +/- 0.038


## Metabric dataset

In [6]:
import pandas as pd

df, X_real, T_real, Y_real, time_horizons = get_dataset("metabric")
df = df[df["duration"] > 0]

X = df.drop(columns=["event", "duration"])
E = df["event"]
T = df["duration"]

X

,x0,x1,x2,x3,x4,x5,x6,x7,x8
0,5.603834,7.811392,10.797988,5.967607,1.0,1.0,0.0,1.0,56.840000
1,5.284882,9.581043,10.204620,5.664970,1.0,0.0,0.0,1.0,85.940002
2,5.920251,6.776564,12.431715,5.873857,0.0,1.0,0.0,1.0,48.439999
3,6.654017,5.341846,8.646379,5.655888,0.0,0.0,0.0,0.0,66.910004
4,5.456747,5.339741,10.555724,6.008429,1.0,0.0,0.0,1.0,67.849998
...,...,...,...,...,...,...,...,...,...
1899,5.946987,5.370492,12.345780,5.741395,1.0,1.0,0.0,1.0,76.839996
1900,5.339228,5.408853,12.176101,5.693043,1.0,1.0,0.0,1.0,63.090000
1901,5.901610,5.272237,14.200950,6.139390,0.0,0.0,0.0,1.0,57.770000
1902,6.818109,5.372744,11.652624,6.077852,1.0,0.0,0.0,1.0,58.889999


In [7]:
metrics = benchmark_tte_reconstruction("metabric", X, T, E)

In [8]:
metrics

,Model,TE Error L1(OOD),TE Error L2(OOD),C-Index(OOD),TE Error L1(ID),TE Error L2(ID),C-Index(ID)
0,tenn,2.5432 +/- 0.1283,2.9658 +/- 0.0924,0.6438 +/- 0.0058,2.4032 +/- 0.0546,2.7821 +/- 0.0616,0.6953 +/- 0.0017
1,date,2.6023 +/- 0.1169,3.0505 +/- 0.0999,0.6358 +/- 0.0061,2.4929 +/- 0.0599,2.9024 +/- 0.0639,0.6636 +/- 0.0028
2,weibull_aft,2.5454 +/- 0.1576,3.0802 +/- 0.2123,0.6342 +/- 0.008,2.4239 +/- 0.0775,2.9378 +/- 0.0902,0.6412 +/- 0.0039
3,cox_ph,2.4971 +/- 0.1437,3.0196 +/- 0.1941,0.6341 +/- 0.008,2.3795 +/- 0.0849,2.8821 +/- 0.0992,0.641 +/- 0.0039
4,random_survival_forest,2.7055 +/- 0.1375,3.2256 +/- 0.1932,0.6349 +/- 0.0143,1.5792 +/- 0.0559,1.9079 +/- 0.058,0.8636 +/- 0.0016
5,survival_xgboost,2.3365 +/- 0.0964,2.7161 +/- 0.1166,0.6448 +/- 0.0132,2.0903 +/- 0.0795,2.4302 +/- 0.0882,0.7174 +/- 0.0016
6,deephit,2.169 +/- 0.0633,2.6182 +/- 0.1026,0.6008 +/- 0.028,2.0492 +/- 0.137,2.4848 +/- 0.1243,0.6156 +/- 0.0169


## Support dataset

In [9]:
import pandas as pd

df, X_real, T_real, Y_real, time_horizons = get_dataset("support")

X = df.drop(columns=["event", "duration"])
E = df["event"]
T = df["duration"]

X

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
0,82.709961,1.0,2.0,1.0,0.0,0.0,0.0,160.0,55.0,16.0,38.195309,142.0,19.000000,1.099854
1,79.660950,1.0,0.0,1.0,0.0,0.0,1.0,54.0,67.0,16.0,38.000000,142.0,10.000000,0.899902
2,23.399990,1.0,2.0,3.0,0.0,0.0,1.0,87.0,144.0,45.0,37.296879,130.0,5.199219,1.199951
3,53.075989,1.0,4.0,3.0,0.0,0.0,0.0,55.0,100.0,18.0,36.000000,135.0,8.699219,0.799927
4,71.794983,0.0,1.0,1.0,0.0,0.0,0.0,65.0,135.0,40.0,38.593750,146.0,0.099991,0.399963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8868,81.064941,0.0,4.0,1.0,0.0,0.0,1.0,111.0,110.0,34.0,39.593750,135.0,13.000000,1.500000
8869,72.560966,0.0,2.0,1.0,0.0,0.0,1.0,53.0,74.0,28.0,34.695309,139.0,7.899414,1.899902
8870,63.228001,0.0,1.0,1.0,0.0,0.0,2.0,95.0,110.0,22.0,38.695309,132.0,7.799805,1.500000
8871,75.405937,0.0,2.0,1.0,1.0,0.0,2.0,109.0,110.0,30.0,36.195309,140.0,15.398438,0.899902


In [10]:
metrics = benchmark_tte_reconstruction("support", X, T, E)

In [11]:
metrics

,Model,TE Error L1(OOD),TE Error L2(OOD),C-Index(OOD),TE Error L1(ID),TE Error L2(ID),C-Index(ID)
0,tenn,3.0254 +/- 0.1645,3.3818 +/- 0.1482,0.6106 +/- 0.0074,3.0802 +/- 0.0925,3.4088 +/- 0.0894,0.649 +/- 0.0041
1,date,3.1213 +/- 0.0811,3.5609 +/- 0.0721,0.5995 +/- 0.002,3.2018 +/- 0.0092,3.6349 +/- 0.018,0.6074 +/- 0.0034
2,weibull_aft,2.9567 +/- 0.0499,3.3238 +/- 0.0631,0.5686 +/- 0.0052,2.9499 +/- 0.0078,3.3167 +/- 0.0021,0.571 +/- 0.0024
3,cox_ph,3.0469 +/- 0.044,3.3896 +/- 0.0548,0.5699 +/- 0.0052,3.0399 +/- 0.0152,3.3821 +/- 0.0056,0.5724 +/- 0.0024
4,random_survival_forest,2.8316 +/- 0.0518,3.2946 +/- 0.0542,0.6093 +/- 0.0054,1.8035 +/- 0.0141,2.0966 +/- 0.0151,0.8237 +/- 0.0016
5,survival_xgboost,2.1075 +/- 0.0257,2.5331 +/- 0.0306,0.6083 +/- 0.0037,1.9017 +/- 0.0138,2.2903 +/- 0.0106,0.6869 +/- 0.0006
6,deephit,2.5472 +/- 0.0322,2.9769 +/- 0.0998,0.5517 +/- 0.0016,2.5266 +/- 0.0274,2.9532 +/- 0.1092,0.5554 +/- 0.005


## flchain dataset

In [4]:
import pandas as pd

df, X_real, T_real, Y_real, time_horizons = get_dataset("flchain")

X = df.drop(columns=["event", "duration"])
E = df["event"]
T = df["duration"]

X

,age,chapter,creatinine,flc.grp,kappa,lambda,mgus,sample.yr,sex
0,97.0,1,1.7,1,5.700,4.860,0,2,0
1,92.0,12,0.9,0,0.870,0.683,0,5,0
2,94.0,1,1.4,1,4.360,3.850,0,2,0
3,92.0,1,1.0,9,2.420,2.220,0,1,0
4,93.0,1,1.1,6,1.320,1.690,0,1,0
...,...,...,...,...,...,...,...,...,...
7869,52.0,16,1.0,6,1.210,1.610,0,0,0
7870,52.0,16,0.8,0,0.858,0.581,0,4,0
7871,54.0,16,0.0,8,1.700,1.720,0,7,0
7872,53.0,16,0.0,9,1.710,2.690,0,0,0


In [5]:
metrics = benchmark_tte_reconstruction("flchain", X, T, E)

In [6]:
metrics

,Model,TE Error L1(OOD),TE Error L2(OOD),C-Index(OOD),TE Error L1(ID),TE Error L2(ID),C-Index(ID)
0,tenn,2.1696 +/- 0.0055,2.592 +/- 0.0149,0.9321 +/- 0.0006,2.0107 +/- 0.0214,2.4135 +/- 0.0202,0.9407 +/- 0.0005
1,date,2.2598 +/- 0.0911,2.6971 +/- 0.1123,0.9273 +/- 0.0009,2.2202 +/- 0.0652,2.6546 +/- 0.1171,0.929 +/- 0.0018
2,weibull_aft,3.0174 +/- 0.0549,3.7565 +/- 0.0831,0.8972 +/- 0.0037,3.0032 +/- 0.0356,3.7405 +/- 0.0488,0.8975 +/- 0.0012
3,cox_ph,3.0422 +/- 0.0523,3.7907 +/- 0.0823,0.8968 +/- 0.0032,3.0278 +/- 0.0335,3.7737 +/- 0.0481,0.897 +/- 0.0012
4,random_survival_forest,2.1113 +/- 0.0339,2.5101 +/- 0.0463,0.9322 +/- 0.002,1.44 +/- 0.0124,1.7144 +/- 0.0146,0.9704 +/- 0.0008
5,survival_xgboost,2.1587 +/- 0.0408,2.5544 +/- 0.0831,0.9323 +/- 0.003,2.0606 +/- 0.0124,2.4395 +/- 0.0188,0.938 +/- 0.0013
6,deephit,4.16 +/- 0.2303,5.0282 +/- 0.2037,0.9241 +/- 0.0021,4.0934 +/- 0.1283,4.9652 +/- 0.1121,0.925 +/- 0.002


## gbsg2 dataset

In [7]:
import pandas as pd

df, X_real, T_real, Y_real, time_horizons = get_dataset("gbsg2")

X = df.drop(columns=["event", "duration"])
E = df["event"]
T = df["duration"]

X

,age,estrec,horTh,menostat,pnodes,progrec,tgrade,tsize
0,70.0,66.0,0,0,3.0,48.0,1,21.0
1,56.0,77.0,1,0,7.0,61.0,1,12.0
2,58.0,271.0,1,0,9.0,52.0,1,35.0
3,59.0,29.0,1,0,4.0,60.0,1,17.0
4,73.0,65.0,0,0,1.0,26.0,1,35.0
...,...,...,...,...,...,...,...,...
681,49.0,84.0,0,1,3.0,1.0,2,30.0
682,53.0,0.0,1,0,17.0,0.0,2,25.0
683,51.0,0.0,0,1,5.0,43.0,2,25.0
684,52.0,34.0,0,0,3.0,15.0,1,23.0


In [8]:
metrics = benchmark_tte_reconstruction("gbsg2", X, T, E)

In [9]:
metrics

,Model,TE Error L1(OOD),TE Error L2(OOD),C-Index(OOD),TE Error L1(ID),TE Error L2(ID),C-Index(ID)
0,tenn,2.677 +/- 0.1498,3.0665 +/- 0.1642,0.6891 +/- 0.0263,2.5254 +/- 0.0821,2.8736 +/- 0.0837,0.7265 +/- 0.0131
1,date,2.6777 +/- 0.0701,3.096 +/- 0.098,0.6862 +/- 0.0121,2.6056 +/- 0.0851,2.983 +/- 0.0877,0.6976 +/- 0.0075
2,weibull_aft,3.1012 +/- 0.2611,3.5418 +/- 0.2654,0.6732 +/- 0.023,2.9836 +/- 0.1108,3.4185 +/- 0.1009,0.6882 +/- 0.0111
3,cox_ph,3.1374 +/- 0.2871,3.5705 +/- 0.2871,0.6727 +/- 0.0232,3.0141 +/- 0.0945,3.4447 +/- 0.0841,0.6885 +/- 0.0109
4,random_survival_forest,2.6346 +/- 0.1222,3.0541 +/- 0.1654,0.6899 +/- 0.0118,1.7488 +/- 0.2055,2.0452 +/- 0.2462,0.8903 +/- 0.01
5,survival_xgboost,2.6174 +/- 0.0717,2.9597 +/- 0.1064,0.6894 +/- 0.0108,2.4831 +/- 0.2728,2.8029 +/- 0.2912,0.7363 +/- 0.0052
6,deephit,2.2673 +/- 0.3622,2.6645 +/- 0.2989,0.659 +/- 0.0014,2.2159 +/- 0.448,2.561 +/- 0.4131,0.6788 +/- 0.0299
